In [14]:
#Since simple movie recommender system suffers so many limitation so I'm going to built a content based recommender
#which will be based on overviews and tagline(description)
# I'll use TF-IDF to colculate similarities between their contents
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [15]:
md = pd. read_csv('data/movies_metadata.csv')
links_small = pd.read_csv('data/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [16]:
md = md.drop([19730, 29503, 35587])

In [17]:
#Check EDA Notebook for how and why I got these indices.
md['id'] = md['id'].astype('int')

In [18]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 24)

In [19]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

/home/vs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/vs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/vs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [20]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [21]:
#check shape of matrix
tfidf_matrix.shape


(9099, 268124)

In [22]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [23]:
cosine_sim[0]


array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [24]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [25]:
#function definition that takes in movie title as input and output similar movies
def get_recommendations(title):
    #get the index of the movie "title"
    idx = indices[title]
    #calculate cosine similarity of that movie with all other movies
    sim_scores = list(enumerate(cosine_sim[idx]))
    #sort the movie based on similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #get the score of 30 most similar movies
    sim_scores = sim_scores[1:31]
    #get the indeces and return the similar movie
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [26]:
#Getting recommendation of movies similar to 'The Godfather'
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object